# Multiple star systems

One of the signature capabilities of **isochrones** is the ability to fit multiple star systems to observational data.  This works by providing a `StarModel` with more detailed information about the observational data, and about how many stars you wish to fit.  There are several layers of potential intricacy here, which we will walk through in stages.

## Unresolved multiple system

Often it is of interest to know what potential binary star configurations are consistent with observations of a star.  For most stars the best available observational data is a combination of broadband magnitudes from various all-sky catalogs and parallax measurements from *Gaia*.  Let's first generate synthetic observations of such a star, and then see what we can recover with a binary star model, and also what inference under a single star model would tell us.

First, we will initialize the isochrone interpolator.  Note that we actually *require* the isochrone interpolator here, rather than the evolution track interpolator, because the model requires the primary and secondary components to have the same age, so that age must be a sampling paramter.

In [1]:
from isochrones import get_ichrone

mist = get_ichrone('mist')

Now, define the "true" system parameters and initialize the `StarModel` accordingly, with two model stars.  Remember that even though we need to use an isochrone interpolator to fit the model, we have to use the evolution tracks to generate synthetic data; this here shows that you can actually do this by using the `.track` complementary attribute.  Note also the use of the utility function `addmags` to combine the magnitudes of the two stars.

In [2]:
from isochrones import StarModel
from isochrones.utils import addmags

distance = 500  # pc
AV = 0.2
mass_A = 1.0
mass_B = 0.5
age = 9.6
feh = 0.0

# Synthetic 2MASS and Gaia magnitudes
bands = ['J', 'H', 'K', 'BP', 'RP', 'G']
props_A = mist.track.generate(mass_A, age, feh, distance=distance, AV=AV, 
                              bands=bands, return_dict=True, accurate=True)
props_B = mist.track.generate(mass_B, age, feh, distance=distance, AV=AV, 
                              bands=bands, return_dict=True, accurate=True)

unc = dict(J=0.02, H=0.02, K=0.02, BP=0.002, RP=0.002, G=0.001)
mags_tot = {b: (addmags(props_A[b], props_B[b]), unc[b]) for b in bands}

# Gaia parallax in mas for a system at 500 pc
parallax = (2, 0.05)

mod_binary = StarModel(mist, **mags_tot, parallax=parallax, N=2, name='demo_binary')

Note here that the only difference from defining the single `StarModel` in previous examples is passing the `N=2` keyword.  This means that the model now has two stars that is part of the model, and that they both contribute to the likelihood of the observed broadband magnitude data.  The structure of the model can be visualized as follows:

In [3]:
mod_binary.print_ascii()

root
 ╚═  J=(12.11, 0.02) @(0.00, 0 [99.00])
    ╚═  H=(11.74, 0.02) @(0.00, 0 [99.00])
       ╚═  K=(11.68, 0.02) @(0.00, 0 [99.00])
          ╚═  BP=(13.61, 0.00) @(0.00, 0 [99.00])
             ╚═  RP=(12.72, 0.00) @(0.00, 0 [99.00])
                ╚═  G=(13.25, 0.00) @(0.00, 0 [99.00])
                   ╠═ 0_0, parallax=(2, 0.05)
                   ╚═ 0_1, parallax=(2, 0.05)


Inspecting this tree to make sure it accurately represents the desired model becomes more important if the model is more complicated, but this simple case is a good example to review.  Each node named with a bandpass represents an observation, with some magnitude and uncertainty (at some separatrion and position angle---irrelevant for the unresolved case).  The model nodes here are named `0_0` and `0_1`, with the first index representing the system, and the second index the star number within that system.  All stars in the same system share the same age, metallicity, distance, and extinction.  In the computation of the likelihood, the apparent magnitude in each observed node is compared with a model-based magnitude that is computed from the *sum of the fluxes of all model nodes underneath that observed node in the tree*.  In the unresolved case, this is trivial, but this structure becomes important when a binary is resolved.  This model, because the two model stars share all attributes except mass, has the following parameters:

In [4]:
mod_binary.param_names

['eep_0_0', 'eep_0_1', 'age_0', 'feh_0', 'distance_0', 'AV_0']

Let's test out the posterior computation, and then run a fit to see if we can recover the true parameters.

In [5]:
pars = [350, 300, 9.7, 0.0, 300, 0.1]
print(mod_binary.lnpost(pars))
%timeit mod_binary.lnpost(pars)

-645762.0097735347
1000 loops, best of 3: 440 µs per loop


In [ ]:
mod_binary.fit()

In [ ]:
%matplotlib inline

columns = ['mass_0_0', 'mass_0_1', 'age_0', 'feh_0', 'distance_0', 'AV_0']
truths = [mass_A, mass_B, age, feh, distance, AV]
mod_binary.corner(columns, truths=truths);

Looks like this recovers the injected parameters pretty well, though not exactly.  It looks like the flat-linear age prior (which weights the fit significantly to older ages) is biasing the masses somewhat low.  Let's explore what happens if we change the prior and try again.

In [ ]:
from isochrones.priors import FlatPrior
mod_binary_2 = StarModel(mist, **mags_tot, parallax=parallax, N=2, name='demo_binary_2')
mod_binary_2.set_prior(age=FlatPrior((8, 10)))
mod_binary_2.fit()

In [ ]:
mod_binary_2.corner(columns, truths=truths);

Curious!  It finds a multimodel distribution that seems to straddle the true parameters.  Very curious.

In [ ]:
eep_A = mist.track.get_eep(mass_A, age, feh, accurate=True)
eep_B = mist.track.get_eep(mass_B, age, feh, accurate=True)
true_pars = [eep_A, eep_B, age, feh, distance, AV]
mod_binary_2.lnprior(true_pars), mod_binary_2.lnlike(true_pars)

In [ ]:
true_pars

In [ ]:
mod_binary_2.corner_physical();

OK, what about a soft prior around the true age?

In [1]:
from isochrones.priors import GaussianPrior

mod_binary_3 = StarModel(mist, **mags_tot, parallax=parallax, N=2, name='demo_binary_3')
mod_binary_3.set_prior(age=GaussianPrior(9.6, 1))

NameError: name 'StarModel' is not defined

In [ ]:
mod_binary_3.fit()

In [ ]:
trial_pars = [320, 260, 9.0, 0.0, 500, 0.2]

In [ ]:
mod_binary_2.lnprior(true_pars), mod_binary_2.lnlike(true_pars)

In [ ]:
mod_binary_2.lnprior(trial_pars), mod_binary_2.lnlike(trial_pars)